In [1]:
import pandas as pd
import numpy as np
import collections
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import math 

from mozo import num_to_range_categories, merge_with_features, how_many_levels, compare_and_combine, compare_and_combine_2, num_to_range_categories, stratified_sample

In [2]:
train = pd.read_hdf('~/pml7/module3/input/train.car_price.h5')
test = pd.read_hdf('~/pml7/module3/input/test.car_price.h5') #zbiór testowy bez odpowiedzi

In [2]:
train = pd.read_hdf('output/newest_train.car_price.h5')
test = pd.read_hdf('output/newest_test.car_price.h5') #zbiór testowy bez odpowiedzi

In [3]:
train.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'pca_30', 'pca_31', 'pca_32', 'pca_33', 'pca_34', 'pca_35', 'pca_36',
       'pca_37', 'pca_38', 'pca_39'],
      dtype='object', length=344)

In [4]:
train['seller_address'].describe()

count                 106385
unique                 11405
top       Radom, Mazowieckie
freq                     657
Name: seller_address, dtype: object

In [5]:
train['seller_address'].value_counts()

Radom, Mazowieckie                                           657
Lublin, Lubelskie                                            523
Kielce, Świętokrzyskie                                       499
Ostrów Mazowiecka, ostrowski, Mazowieckie                    350
Bydgoszcz, Kujawsko-pomorskie                                346
                                                            ... 
Gronowo Elbląskie, elbląski, Warmińsko-mazurskie               1
Skryta 1 - 63-600 Kępno, kępiński, Wielkopolskie (Polska)      1
Szczepanki, brodnicki, Kujawsko-pomorskie                      1
Bolesławice, bolesławiecki, Dolnośląskie                       1
Kostów, kluczborski, Opolskie                                  1
Name: seller_address, Length: 11405, dtype: int64

In [ ]:
Wielkopolskie
Kujawsko-pomorskie
Małopolskie
Łódzkie
Dolnośląskie
Lubelskie
Lubuskie
Mazowieckie
Opolskie
Podlaskie
Pomorskie
Śląskie
Podkarpackie
Świętokrzyskie
Warmińsko-Mazurskie
Zachodniopomorskie

In [28]:
total = pd.concat([train, test], ignore_index=True)

In [29]:
def get_wojewodzctwo(adres):
    if adres is None: return '-1'
    if adres.lower().find("wielkopolsk") >= 0: return 'Wielkopolskie'
    if adres.lower().find("kujawsk") >= 0: return 'Kujawsko-pomorskie'    
    if adres.lower().find("małopolsk") >= 0: return 'Małopolskie'
    if adres.lower().find("malopolsk") >= 0: return 'Małopolskie'
    if adres.lower().find("łódź") >= 0: return 'Łódzkie'
    if adres.lower().find("łódzki") >= 0: return 'Łódzkie'
    if adres.lower().find("lodzki") >= 0: return 'Łódzkie'
    if adres.lower().find("dolnośląsk") >= 0: return 'Dolnośląskie'
    if adres.lower().find("dolnoslask") >= 0: return 'Dolnośląskie'
    if adres.lower().find("dolny śląsk") >= 0: return 'Dolnośląskie'
    if adres.lower().find("lubels") >= 0: return 'Lubelskie'
    if adres.lower().find("lubusk") >= 0: return 'Lubuskie'
    if adres.lower().find("mazow") >= 0: return 'Mazowieckie'
    if adres.lower().find("opole") >= 0: return 'Opolskie'
    if adres.lower().find("opolskie") >= 0: return 'Opolskie'
    if adres.lower().find("podlask") >= 0: return 'Podlaskie'
    if adres.lower().find("zachodniopomorsk") >= 0: return 'Zachodniopomorskie'
    if adres.lower().find("pomorskie") >= 0: return 'Pomorskie'
    if adres.lower().find("slaskie") >= 0: return 'Śląskie'
    if adres.lower().find("śląskie") >= 0: return 'Śląskie'
    if adres.lower().find("śląsk") >= 0: return 'Śląskie'
    if adres.lower().find("podkarpac") >= 0: return 'Podkarpackie'
    if adres.lower().find("świętokrzyskie") >= 0: return 'Świętokrzyskie'
    if adres.lower().find("swietokrzyskie") >= 0: return 'Świętokrzyskie'
    if adres.lower().find("warmi") >= 0: return 'Warmińsko-Mazurskie'
    if adres.lower().find("mazur") >= 0: return 'Warmińsko-Mazurskie'
    return '-1'
        
get_wojewodzctwo('małopolsk')

'Małopolskie'

In [32]:
total['seller_wojewodzctwo'] = total.apply(lambda row: get_wojewodzctwo(row['seller_address']), axis=1)
total['seller_wojewodzctwo'].value_counts()


Mazowieckie            22620
Wielkopolskie          22125
-1                     19209
Śląskie                16155
Małopolskie            12155
Dolnośląskie           10411
Łódzkie                 9698
Pomorskie               7436
Lubelskie               7088
Kujawsko-pomorskie      6970
Podkarpackie            5105
Świętokrzyskie          5021
Zachodniopomorskie      4734
Warmińsko-Mazurskie     3767
Lubuskie                3229
Opolskie                2800
Podlaskie               2093
Name: seller_wojewodzctwo, dtype: int64

In [38]:
def categorial_to_percent_range(df, var_cat, n_cat):
    counts = pd.DataFrame(df[var_cat].value_counts()).reset_index()
    
    def get_level(kod):
        x = counts.loc[counts['index'] == kod, [var_cat]]
        if x.empty:
            return -1
        else: 
            return x.values[0][0]
        
    df['encoded_' + var_cat] = df.apply(lambda row: get_level(row[var_cat]), axis=1)
    print(df['encoded_' + var_cat].value_counts())
    
    num_to_range_categories(df, 'encoded_' + var_cat, n_cat=n_cat) 
    df['percent_range_' + var_cat] = df.apply(lambda row: int(row['range_'+('encoded_' + var_cat)+'_cat'].replace('perc_', '')), axis=1)
    print(total['percent_range_' + var_cat].value_counts() )


In [40]:
categorial_to_percent_range(total, 'seller_wojewodzctwo', 5)

22620    22620
22125    22125
19209    19209
16155    16155
12155    12155
10411    10411
9698      9698
7436      7436
7088      7088
6970      6970
5105      5105
5021      5021
4734      4734
3767      3767
3229      3229
2800      2800
2093      2093
Name: encoded_seller_wojewodzctwo, dtype: int64
perc_60     35364
perc_20     31192
perc_00     26749
perc_100    22620
perc_40     22566
perc_80     22125
Name: range_encoded_seller_wojewodzctwo_cat, dtype: int64
['perc_00', 'perc_100', 'perc_20', 'perc_40', 'perc_60', 'perc_80']
0
60     35364
20     31192
0      26749
100    22620
40     22566
80     22125
Name: percent_range_seller_wojewodzctwo, dtype: int64


In [41]:
filter_col = ['car_id', 
    'seller_wojewodzctwo', 'encoded_seller_wojewodzctwo', 'percent_range_seller_wojewodzctwo'
]

In [42]:
total.loc[~total['price_value'].isnull(), filter_col].to_csv('output/ft_new_param_4_train.csv', index=False) 
total.loc[total['price_value'].isnull(), filter_col].to_csv('output/ft_new_param_4_test.csv', index=False)